# CS295/395: Secure Distributed Computation
## In-Class Exercise, 10/12/2020

In [ ]:
# Imports and definitions
import numpy as np
from collections import defaultdict
from collections import namedtuple
import urllib.request

# Utilities for Primes

Taken from: https://github.com/mikeivanov/paillier/blob/master/paillier/primes.py

In [ ]:
import random
import sys
import math

def ipow(a, b, n):
    """calculates (a**b) % n via binary exponentiation, yielding itermediate
       results as Rabin-Miller requires"""
    A = a = a % n
    yield A
    t = 1
    while t <= b:
        t <<= 1

    # t = 2**k, and t > b
    t >>= 2
    
    while t:
        A = (A * A) % n
        if t & b:
            A = (A * a) % n
        yield A
        t >>= 1

def rabin_miller_witness(test, possible):
    """Using Rabin-Miller witness test, will return True if possible is
       definitely not prime (composite), False if it may be prime."""    
    return 1 not in ipow(test, possible-1, possible)

smallprimes = (2,3,5,7,11,13,17,19,23,29,31,37,41,43,
               47,53,59,61,67,71,73,79,83,89,97)

def default_k(bits):
    return int(max(40, 2 * bits))

def is_probably_prime(possible, k=None):
    if possible == 1:
        return True
    if k is None:
        k = default_k(possible.bit_length())
    for i in smallprimes:
        if possible == i:
            return True
        if possible % i == 0:
            return False
    for i in range(k):
        test = random.randrange(2, possible - 1) | 1
        if rabin_miller_witness(test, possible):
            return False
    return True

def generate_prime(bits, k=None):
    """Will generate an integer of b bits that is probably prime 
       (after k trials). Reasonably fast on current hardware for 
       values of up to around 512 bits."""    
    assert bits >= 8

    if k is None:
        k = default_k(bits)

    while True:
        possible = random.randrange(2 ** (bits-1) + 1, 2 ** bits) | 1
        if is_probably_prime(possible, k):
            return possible

In [ ]:
generate_prime(16)

# The Paillier Cryptosystem

Taken from: https://github.com/mikeivanov/paillier/blob/master/paillier/paillier.py

In [ ]:
def invmod(a, p, maxiter=1000000):
    """The multiplicitive inverse of a in the integers modulo p:
         a * b == 1 mod p
       Returns b.
       (http://code.activestate.com/recipes/576737-inverse-modulo-p/)"""
    if a == 0:
        raise ValueError('0 has no inverse mod %d' % p)
    r = a
    d = 1
    for i in range(min(p, maxiter)):
        d = ((p // r + 1) * d) % p
        r = (d * a) % p
        if r == 1:
            break
    else:
        raise ValueError('%d has no inverse mod %d' % (a, p))
    return d

def modpow(base, exponent, modulus):
    """Modular exponent:
         c = b ^ e mod m
       Returns c.
       (http://www.programmish.com/?p=34)"""
    result = 1
    while exponent > 0:
        if exponent & 1 == 1:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class PrivateKey(object):

    def __init__(self, p, q, n):
        self.l = (p-1) * (q-1)
        self.m = invmod(self.l, n)

    def __repr__(self):
        return '<PrivateKey: %s %s>' % (self.l, self.m)

class PublicKey(object):

    @classmethod
    def from_n(cls, n):
        return cls(n)

    def __init__(self, n):
        self.n = n
        self.n_sq = n * n
        self.g = n + 1

    def __repr__(self):
        return '<PublicKey: %s>' % self.n

def generate_keypair(bits):
    p = generate_prime(bits / 2)
    q = generate_prime(bits / 2)
    n = p * q
    return PrivateKey(p, q, n), PublicKey(n)

def encrypt(pub, plain):
    while True:
        r = generate_prime(round(math.log(pub.n, 2)))
        if r > 0 and r < pub.n:
            break
    x = pow(r, pub.n, pub.n_sq)
    cipher = (pow(pub.g, plain, pub.n_sq) * x) % pub.n_sq
    return cipher

def decrypt(priv, pub, cipher):
    x = pow(cipher, priv.l, pub.n_sq) - 1
    plain = ((x // pub.n) * priv.m) % pub.n
    return plain

In [ ]:
# Example: encryption followed by decryption
sk, pk = generate_keypair(128)
ct = encrypt(pk, 50)
decrypt(sk, pk, ct)

In [ ]:
# Example: encrypting the same thing twice yields different results
sk, pk = generate_keypair(16)
ct1 = encrypt(pk, 50)
ct2 = encrypt(pk, 50)
ct1, ct2

# Questions

## Question 1

Define addition for ciphertexts.

In [ ]:
def e_add(pub, a, b):
    """Add one encrypted integer to another"""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# TEST CASE
sk, pk = generate_keypair(128)
ct1 = encrypt(pk, 50)
ct2 = encrypt(pk, 10)
ct3 = e_add(pk, ct1, ct2)

assert decrypt(sk, pk, ct3) == 60

## Question 2

Define addition-by-constant for ciphertexts.

In [ ]:
def e_add_const(pub, a, n):
    """Add constant n to an encrypted integer"""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# TEST CASE
sk, pk = generate_keypair(128)
ct1 = encrypt(pk, 50)
ct3 = e_add_const(pk, ct1, 10)

assert decrypt(sk, pk, ct3) == 60

## Question 3

Define multiplication-by-constant for ciphertexts.

In [ ]:
def e_mul_const(pub, a, n):
    """Multiplies an ancrypted integer by a constant"""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# TEST CASE
sk, pk = generate_keypair(128)
ct1 = encrypt(pk, 50)
ct3 = e_mul_const(pk, ct1, 3)

assert decrypt(sk, pk, ct3) == 150